#### Check all events are present in Open-Ephys output

Iterates through all sessions of an animal and prints the unique labels for events in continuous.dat file in 'NI-DAQmx-103.PXIe-6341' data stream. '1' denotes the heartbeat pulse every second (duration 0.5 seconds), and '4' denotes a TTL pulse

In [ ]:
import os
import glob
import spikeinterface.extractors as se
import pandas as pd
import numpy as np

# Define variables
input_root_dir = "W:\\projects\\FlexiVexi\\behavioural_data"
animal_ID = 'FNT098'
kilosort_subfolder = 'intermediate_variables'

# List all the names of all subfolders within animal folder
animal_folder = os.path.join(input_root_dir, animal_ID)
session_list = [f.path for f in os.scandir(animal_folder) if f.is_dir()]

# Iterate through all sessions in session_list and return session folder and unique labels in event_info_PXI
for session_folder in session_list:

    # Get path to Open-Ephys Record Node within session folder
    matching_files = glob.glob(os.path.join(session_folder, '**', 'settings.xml'), recursive=True)
    if matching_files:

        # Get the first matching file
        first_matching_file = matching_files[0]

        # Get the directory of the first matching file
        path_to_recording = os.path.dirname(first_matching_file)

        # Read open ephys event times
        event = se.read_openephys_event(folder_path=path_to_recording)

        # Get PXI channel ID
        PXI_channel_ID = event.channel_ids[1]

        # Get event times and event info for the PXI channel
        event_times_PXI = event.get_event_times(channel_id=PXI_channel_ID)
        event_info_PXI = event.get_events(channel_id=PXI_channel_ID)

        # Get unique values in event_info_PXI labels
        unique_labels = np.unique(event_info_PXI['label'])
        print(session_folder)
        print('Unique labels: ' + str(unique_labels))

        # Append unique labels and session ID to pandas data frame
        #df = df.concat(df, {'session_ID': os.path.basename(session_folder), 'unique_labels': unique_labels}, ignore_index=True)